## Deploy endpoint with the trained data

After model training, we will deploy an endpoint with trained model. But before doing that pleanse complete step 1 and 2.

In [ ]:
%store -r

In [ ]:
import os
import boto3
import sagemaker as sm
print(f'SageMaker:{sm.__version__}')
from sagemaker import get_execution_role
from sagemaker.pytorch import PyTorchModel
from sagemaker.predictor import Predictor, json_serializer, json_deserializer

In [ ]:
class JSONPredictor(Predictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super(JSONPredictor, self).__init__(endpoint_name, sagemaker_session, json_serializer, json_deserializer)

In [ ]:
env = {
    'SAGEMAKER_MODEL_SERVER_WORKERS': '1'
}

print(f'Use model {repackged_model_path}')

sagemakerSession = sm.session.Session(boto3.session.Session(region_name=current_region))
fd_sl_model = PyTorchModel(model_data=repackged_model_path, 
                           role=sagemaker_exec_role,
                           entry_point='./FD_SL_DGL/code/fd_sl_deployment_entry_point.py',
                           framework_version='1.6.0',
                           py_version='py3',
                           predictor_cls=JSONPredictor,
                           env=env,
                           sagemaker_session=sagemakerSession)

In [ ]:
fd_sl_predictor = fd_sl_model.deploy(instance_type='ml.c5.4xlarge', 
                                     initial_instance_count=1,
                                    )

In [ ]:
endpoint = fd_sl_predictor.endpoint_name
print(endpoint)

### Test the inference endpoint

Using the script locates at `./FD_SL_DGL/clients_python/` to test the inference endpoint.

In [ ]:
! export AWS_DEFAULT_REGION=$current_region && cd ./FD_SL_DGL/clients_python/ && python ./client_boto_demo.py --endpoint_name $endpoint --test_round 10

### Cleanup the resources

In [ ]:
fd_sl_predictor.delete_endpoint()
fd_sl_model.delete_model()